# Tech Challenge - Análise de Atrasos de Voos

**O problema** </br>
O transporte aéreo é uma parte vital da infraestrutura global, mas os 
atrasos de voos impactam milhões de passageiros todos os anos. Neste projeto, 
você utilizará o conjunto de dados público que contém informações detalhadas 
sobre voos nos EUA para desenvolver análises e modelos preditivos e/ou 
exploratórios aplicando técnicas de Machine Learning supervisionado e não 
supervisionado. 

**Objetivos do Projeto** </br>
Aplicar o conhecimento adquirido sobre modelagem supervisionada e não 
supervisionada e desenvolver um pipeline completo de ciência de dados, desde 
a exploração dos dados até a interpretação dos resultados. 

## Exploração dos dados

### Carregando os Dados dos Voos

In [21]:
import pandas as pd

# Carregar os CSVs
flights_df = pd.read_csv('flights.csv')
airlines_df = pd.read_csv('airlines.csv')
airports_df = pd.read_csv('airports.csv')

C:\Users\Carol\AppData\Local\Temp\ipykernel_24360\518823895.py:4: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv('flights.csv')


### Exploração e tratamento dos dados

#### Merge dos dados de companhias aéreas e aeroportos com voos
Iremos relacionar os dados das companhias aéreas e dos aeroportos com a tabela principal de voos para exploração completa dos dados.

In [24]:
df = flights_df.merge(airlines_df, left_on='AIRLINE', right_on='IATA_CODE', how='left', suffixes=('', '_AIRLINE'))
df = df.merge(airports_df, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='left', suffixes=('', '_ORIGIN'))

# Adicionar nome do aeroporto de destino
df = df.merge(airports_df[['IATA_CODE', 'AIRPORT']], left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='left', suffixes=('', '_DEST'))

df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIRLINE_AIRLINE,IATA_CODE_ORIGIN,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,IATA_CODE_DEST,AIRPORT_DEST
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport


Agora com os dados unificados, iremos remover algunas colunas que não influenciam diretamente na análise atraso de voos, portanto, inicialmente, para exploração dos dados, iremos focar em fatores operacionais e temporais como aeroportos de origem/destino, horários e distâncias.

In [26]:
# Remoção de colunas desnecessárias
df = df.drop(columns=['FLIGHT_NUMBER', 'TAIL_NUMBER', 'IATA_CODE_ORIGIN', 'LATITUDE', 'LONGITUDE', 'CANCELLATION_REASON', 'IATA_CODE_DEST', 'IATA_CODE_ORIGIN'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 35 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   ORIGIN_AIRPORT       object 
 6   DESTINATION_AIRPORT  object 
 7   SCHEDULED_DEPARTURE  int64  
 8   DEPARTURE_TIME       float64
 9   DEPARTURE_DELAY      float64
 10  TAXI_OUT             float64
 11  WHEELS_OFF           float64
 12  SCHEDULED_TIME       float64
 13  ELAPSED_TIME         float64
 14  AIR_TIME             float64
 15  DISTANCE             int64  
 16  WHEELS_ON            float64
 17  TAXI_IN              float64
 18  SCHEDULED_ARRIVAL    int64  
 19  ARRIVAL_TIME         float64
 20  ARRIVAL_DELAY        float64
 21  DIVERTED             int64  
 22  CANCELLED            int64  
 23  AIR_SYSTEM_DELAY     float64
 24

### Análise Descritiva Inicial

Antes de avançar para visualizações e modelagem preditiva, realizaremos uma análise descritiva abrangente do dataset unificado. Isso inclui estatísticas resumidas para variáveis numéricas e categóricas, foco específico em atrasos de voos (como percentuais, distribuições e correlações), e uma verificação inicial de valores ausentes. Esses insights nos ajudarão a identificar padrões, outliers e necessidades de tratamento de dados, preparando o terreno para visualizações que tragam insights acionáveis e um tratamento adequado de valores ausentes.

In [ ]:
print("\n" + "="*60)
print("ANÁLISE FOCADA EM ATRASOS E VARIÁVEIS PRINCIPAIS")
print("="*60)

# Estatísticas específicas para atrasos
print(f"\nTotal de voos analisados: {len(df)}")
print(f"Voos com atraso na chegada > 0 min: {(df['ARRIVAL_DELAY'] > 0).sum()} ({((df['ARRIVAL_DELAY'] > 0).sum() / len(df) * 100):.2f}%)")
print(f"Voos com atraso na chegada > 15 min: {(df['ARRIVAL_DELAY'] > 15).sum()} ({((df['ARRIVAL_DELAY'] > 15).sum() / len(df) * 100):.2f}%)")
print(f"Voos com atraso na partida > 15 min: {(df['DEPARTURE_DELAY'] > 15).sum()} ({((df['DEPARTURE_DELAY'] > 15).sum() / len(df) * 100):.2f}%)")

print(f"\nMédia de atraso na chegada: {df['ARRIVAL_DELAY'].mean():.2f} min")
print(f"Mediana de atraso na chegada: {df['ARRIVAL_DELAY'].median():.2f} min")
print(f"Desvio padrão de atraso na chegada: {df['ARRIVAL_DELAY'].std():.2f} min")
print(f"Atraso máximo na chegada: {df['ARRIVAL_DELAY'].max():.2f} min")
print(f"Atraso mínimo na chegada: {df['ARRIVAL_DELAY'].min():.2f} min")

# Percentis para ARRIVAL_DELAY
print(f"\nPercentis de ARRIVAL_DELAY:")
percentis = df['ARRIVAL_DELAY'].quantile([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
for p, v in percentis.items():
    print(f"{p*100:.0f}th percentile: {v:.2f} min")

# Correlação entre atrasos
print("\nCorrelação entre atrasos de partida e chegada:")
corr = df[['ARRIVAL_DELAY', 'DEPARTURE_DELAY']].corr()
print(corr)

# Análise de variáveis categóricas principais
print("\n" + "="*60)
print("ANÁLISE DE VARIÁVEIS CATEGÓRICAS")
print("="*60)
print(f"Companhias aéreas distintas: {df['AIRLINE'].nunique()}")
print(f"Aeroportos de origem distintos: {df['ORIGIN_AIRPORT'].nunique()}")
print(f"Aeroportos de destino distintos: {df['DESTINATION_AIRPORT'].nunique()}")

print("\nTop 5 companhias aéreas por número de voos:")
print(df['AIRLINE_AIRLINE'].value_counts().head(5))

print("\nTop 5 aeroportos de origem por número de voos:")
print(df['AIRPORT'].value_counts().head(5))

# Verificar valores ausentes após limpeza inicial
print("\n" + "="*60)
print("VERIFICAÇÃO DE VALORES AUSENTES")
print("="*60)

print("\nPercentual de valores nulos por coluna:")
print((df.isnull().sum() / len(df) * 100).round(2))


ANÁLISE FOCADA EM ATRASOS E VARIÁVEIS PRINCIPAIS

Total de voos analisados: 5819079
Voos com atraso na chegada > 0 min: 2086896 (35.86%)
Voos com atraso na chegada > 15 min: 1023498 (17.59%)
Voos com atraso na partida > 15 min: 1018558 (17.50%)

Média de atraso na chegada: 4.41 min
Mediana de atraso na chegada: -5.00 min
Desvio padrão de atraso na chegada: 39.27 min
Atraso máximo na chegada: 1971.00 min
Atraso mínimo na chegada: -87.00 min

Percentis de ARRIVAL_DELAY:
10th percentile: -20.00 min
25th percentile: -13.00 min
50th percentile: -5.00 min
75th percentile: 8.00 min
90th percentile: 34.00 min
95th percentile: 66.00 min
99th percentile: 167.00 min

Correlação entre atrasos de partida e chegada:
                 ARRIVAL_DELAY  DEPARTURE_DELAY
ARRIVAL_DELAY         1.000000         0.944672
DEPARTURE_DELAY       0.944672         1.000000

ANÁLISE DE VARIÁVEIS CATEGÓRICAS
Companhias aéreas distintas: 14
Aeroportos de origem distintos: 930
Aeroportos de destino distintos: 930

Top

### Preparação dos dados: limpeza e tratamento de valores ausentes

Vamos começar limpando os dados – lidando com valores ausentes e outliers – e depois criando features derivadas, como o período do dia ou estações do ano, para enriquecer nossa análise e tornar os modelos mais poderosos.

In [ ]:
# Usar flights_df como base para análise
df = flights_df.copy()

# Remover colunas desnecessárias (baseado na análise inicial)
colunas_desnecessarias = ['FLIGHT_NUMBER', 'TAIL_NUMBER', 'CANCELLED', 'CANCELLATION_REASON', 'DIVERTED', 
                          'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df = df.drop(columns=colunas_desnecessarias, errors='ignore')

# Verificar valores nulos antes do tratamento
print("Valores nulos antes do tratamento:")
print(df.isnull().sum())

# Tratamento de valores ausentes
# Para ARR_DELAY (target principal), imputar com 0 (atrasos não reportados como 0) e dropar linhas críticas
df['ARR_DELAY'] = df['ARR_DELAY'].fillna(0)
df = df.dropna(subset=['ARR_DELAY'])  # Garantir que o target não tenha NaN

# Para DEPARTURE_DELAY, imputar com 0 (sem atraso na partida)
df['DEPARTURE_DELAY'] = df['DEPARTURE_DELAY'].fillna(0)

# Dropar linhas com muitos NaN em colunas essenciais (e.g., tempos de voo)
df = df.dropna(subset=['DEPARTURE_TIME', 'ARRIVAL_TIME', 'AIR_TIME'])

# Engenharia de features
# Criar features temporais derivadas
df['FL_DATE'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-' + df['DAY'].astype(str))
df['DEP_HOUR'] = pd.to_datetime(df['DEPARTURE_TIME'], format='%H%M', errors='coerce').dt.hour
df['ARR_HOUR'] = pd.to_datetime(df['ARRIVAL_TIME'], format='%H%M', errors='coerce').dt.hour
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek  # 0=Segunda, 6=Domingo
df['MONTH'] = df['FL_DATE'].dt.month
df['SEASON'] = df['MONTH'].apply(lambda x: 'Winter' if x in [12,1,2] else 'Spring' if x in [3,4,5] else 'Summer' if x in [6,7,8] else 'Fall')

# Criar variável target binária para atrasos (>15 min é considerado atraso)
df['DELAYED'] = (df['ARR_DELAY'] > 15).astype(int)

# Encoding categórico (opcional para modelagem; manter para EDA)
# df = pd.get_dummies(df, columns=['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'], drop_first=True)

# Verificar valores nulos após tratamento
print("\nValores nulos após tratamento:")
print(df.isnull().sum())

# Estatísticas descritivas para validação
print("\nEstatísticas descritivas de ARR_DELAY:")
print(df['ARR_DELAY'].describe())

print(f"\nDataset preparado com {df.shape[0]} linhas e {df.shape[1]} colunas.")

# Visualizando os Insights

Para tornar nossos achados palpáveis, vamos criar visualizações. Como artistas de dados, pintaremos gráficos que contam a história dos atrasos, revelando tendências e anomalias. Cada gráfico é uma janela para o mundo dos voos, ajudando-nos a ver o invisível e transformar dados em conhecimento visual!

In [ ]:
# Placeholder para visualizações
# df['atraso'].hist()
# plt.show()

# Conclusões e Próximos Passos

Chegamos ao fim da nossa jornada. Aqui, resumimos os insights descobertos sobre os atrasos de voos, refletindo sobre o que aprendemos. Como viajantes experientes, pensamos nos próximos passos: como usar esse conhecimento para melhorar o futuro das viagens aéreas. Obrigado por voar conosco nessa análise!

In [ ]:
# Placeholder para conclusões
print("Análise de atrasos de voos concluída!")

In [ ]:
# Placeholder para considerações finais
# print("Análise completa. Próximos passos: integrar dados em tempo real.")
print("Fim da análise!")

# Considerações Finais e Futuras Pesquisas

Encerrando nossa odisseia analítica, refletimos sobre as lições aprendidas e olhamos para o horizonte de futuras explorações. Como filósofos de dados, pensamos em expansões como integração de dados em tempo real ou estudos comparativos globais. Esta jornada nos deixa com uma compreensão mais profunda, prontos para voar ainda mais alto em análises futuras.

In [ ]:
# Placeholder para interpretação de resultados
# # Analisar coeficientes do modelo
# # print(model.coef_)
# print("Resultados interpretados!")

# Interpretação dos Resultados e Insights

Desvendando os mistérios revelados pelos modelos, vamos interpretar os resultados para extrair insights acionáveis. Como tradutores de dados, transformaremos números em narrativas compreensíveis, destacando fatores-chave e recomendações práticas. Cada insight é uma luz no caminho, iluminando o futuro das viagens aéreas com sabedoria derivada de dados.

In [ ]:
# Placeholder para validação de modelos
# from sklearn.metrics import accuracy_score
# # y_pred = model.predict(X_test)
# # print(accuracy_score(y_test, y_pred))
# print("Modelo validado!")

# Validação e Avaliação dos Modelos

Testando a robustez de nossas previsões, vamos validar os modelos com dados não vistos. Como árbitros de precisão, mediremos acurácia, precisão e recall, garantindo que nossas ferramentas sejam confiáveis. Cada teste é um passo na jornada para a excelência, refinando nossos modelos até que sejam dignos de guiar decisões reais.

In [ ]:
# Placeholder para modelagem preditiva
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# # model = LogisticRegression()
# # model.fit(X_train, y_train)
# print("Modelo treinado!")

# Modelagem Preditiva de Atrasos

Como videntes de dados, vamos construir modelos que prevêem a probabilidade de atrasos. Usando algoritmos de machine learning, transformaremos dados históricos em oráculos do futuro. Cada modelo é uma ferramenta poderosa, ajudando passageiros e companhias a antecipar e evitar inconvenientes, tornando as viagens mais previsíveis e agradáveis.

In [ ]:
# Placeholder para análise de causas
# causas = df['causa_atraso'].value_counts()
# causas.plot(kind='pie')
# plt.show()

# Análise Detalhada das Causas de Atrasos

Mergulhando nas raízes dos problemas, vamos dissecar as causas dos atrasos com precisão cirúrgica. Como investigadores forenses, categorizaremos e quantificaremos fatores como clima, manutenção e tráfego aéreo. Cada causa é um capítulo na saga dos voos atrasados, nos equipando com conhecimento para mitigar esses desafios no futuro.

In [ ]:
# Placeholder para impacto de companhias
# df.groupby('companhia')['atraso'].mean().sort_values().plot(kind='bar')
# plt.show()

# Impacto das Companhias Aéreas

Entrando no mundo das companhias aéreas, vamos avaliar como cada uma contribui para o panorama dos atrasos. Como juízes imparciais de dados, compararemos performances, revelando líderes em pontualidade e aqueles que precisam de ajustes. Cada companhia é uma peça no quebra-cabeça dos voos, e nossa análise ajudará a destacar excelências e áreas de melhoria.

In [ ]:
# Placeholder para comparação de aeroportos
# df.groupby('aeroporto_origem')['atraso'].mean().sort_values().plot(kind='bar')
# plt.show()

# Comparação entre Aeroportos

Como exploradores de hubs aéreos, vamos comparar os desempenhos dos aeroportos, descobrindo quais são os mais pontuais e quais enfrentam maiores desafios. Cada aeroporto é um personagem único nessa narrativa, com suas próprias histórias de eficiência e obstáculos. Vamos mapear o território dos voos, identificando hotspots de atrasos e ilhas de pontualidade.

In [ ]:
# Placeholder para análise temporal
# df['data'] = pd.to_datetime(df['data'])
# df.groupby(df['data'].dt.month)['atraso'].mean().plot()
# plt.show()

# Análise de Tendências Temporais

Viajando através do tempo, vamos explorar como os atrasos evoluem ao longo dos dias, meses e anos. Como cronistas de dados, traçaremos linhas no tempo que revelam padrões sazonais, picos de demanda e flutuações imprevisíveis. Cada tendência temporal é uma história contada pelos voos, nos ensinando sobre o ritmo das viagens aéreas e os momentos de maior vulnerabilidade.

In [ ]:
# Placeholder para limpeza de dados
# df.dropna(inplace=True)
# df.drop_duplicates(inplace=True)
# print("Dados limpos!")

# Limpeza e Preparação dos Dados

Como jardineiros cuidadosos em um jardim de dados, agora vamos limpar e preparar o solo para nossas análises. Removeremos impurezas como valores ausentes, duplicatas e inconsistências, transformando dados brutos em um terreno fértil para insights. Cada passo de limpeza é uma jornada de purificação, revelando a verdadeira essência dos atrasos de voos e preparando-nos para descobertas mais profundas.